In [1]:
import json
import numpy as np

In [2]:
blacklist_seccomp = json.load(open('../profiles/default_x86_extended_blacklist.json', 'r'))
blacklist_block = blacklist_seccomp["syscalls"][0]["names"]
blacklist_log = blacklist_seccomp["syscalls"][1]["names"]

def load_from_file(key):
    res = []
    # append runs from ycsb
    if key in ['mongo', 'redis', 'postgres', 'cassandra', 'memcache']:
        for workload in ['a', 'b', 'f']:
            res.append(json.load(open('../results/ycsb/{}_{}.json'.format(key,workload), 'r')))

    # append runs from cloudSuite webServing
    if key in ['elgg', 'memcache', 'mariadb']:
        for workload in ['', '_run', '_manual']:
            res.append(json.load(open('../results/cloudsuite/webServing_{}{}.json'.format(key,workload), 'r')))

    # append runs from cloudSuite dataServing
    if key == 'cassandra':
        for workload in ['', '_load']:
            res.append(json.load(open('../results/cloudsuite/dataServing{}.json'.format(workload), 'r')))

    return res

def get_blocked_by(syscall_arr, compare_list):
    blocked_arr = []

    for syscall in syscall_arr:
        if syscall in compare_list:
            blocked_arr.append({syscall : syscall_arr[syscall]})
    return blocked_arr

def get_blocked_by_blacklist(syscall_arr):
    return get_blocked_by(syscall_arr, blacklist_block)

def get_logged_by_blacklist(syscall_arr):
    return get_blocked_by(syscall_arr, blacklist_log)

In [3]:
captures = load_from_file('mariadb')


In [4]:
aggregated_blocked = {}

for i in range(0, len(captures)):
    b_list = get_blocked_by_blacklist(captures[i])
    l_list = get_logged_by_blacklist(captures[i])

    print("{} blocked : {}".format(len(b_list), b_list))
    print("{} logged : {}".format(len(l_list), l_list))
    print("-----------------")
    for j in range(0, len(b_list)):
        s = list(b_list[j].keys())[0]

        if not (s in aggregated_blocked):
            aggregated_blocked[s] = 0
        aggregated_blocked[s] += b_list[j][s]


1 blocked : [{'restart_syscall': 34}]
12 logged : [{'setpgid': 2}, {'chown': 1}, {'prctl': 108}, {'getpgrp': 2675}, {'execve': 1023}, {'umask': 12}, {'fchown': 135}, {'fchownat': 138}, {'chmod': 1}, {'setgroups': 1}, {'setgid': 1}, {'setuid': 1}]
-----------------
1 blocked : [{'restart_syscall': 137}]
12 logged : [{'getpgrp': 2651}, {'execve': 1023}, {'chown': 1}, {'umask': 12}, {'fchownat': 138}, {'fchown': 135}, {'chmod': 1}, {'prctl': 108}, {'setgroups': 1}, {'setgid': 1}, {'setuid': 1}, {'setpgid': 2}]
-----------------
1 blocked : [{'restart_syscall': 19}]
12 logged : [{'getpgrp': 2494}, {'fchown': 135}, {'fchownat': 138}, {'chmod': 1}, {'execve': 1017}, {'umask': 11}, {'prctl': 108}, {'setgroups': 1}, {'setgid': 1}, {'setuid': 1}, {'setpgid': 2}, {'chown': 1}]
-----------------


In [5]:
for syscall in aggregated_blocked:
    print('{} (mean) {}'.format(syscall, aggregated_blocked[syscall] / 3))

restart_syscall (mean) 63.333333333333336


In [6]:
# aggregate logged
aggregated_logged = {}

for capture_name in ['mongo', 'redis', 'postgres', 'cassandra', 'memcache', 'elgg', 'mariadb']:
    caps = load_from_file(capture_name)

    for cap in caps:
        logged = get_logged_by_blacklist(cap)
        for entity in logged:
            syscall = list(entity.keys())[0]
            amount = entity[syscall]

            if not (syscall in aggregated_logged):
                aggregated_logged[syscall] = 0

            aggregated_logged[syscall] += amount

print(aggregated_logged)

{'execve': 3976, 'chmod': 128, 'prctl': 613756, 'setsid': 86, 'umask': 231, 'getpgrp': 7839, 'fchownat': 418, 'setgroups': 77, 'setgid': 44, 'setuid': 41, 'fchmodat': 12, 'capset': 3, 'chown': 9, 'setpgid': 9, 'fchown': 427}
